In [5]:
import xml.etree.ElementTree as ET
from os import listdir
from os.path import isfile, join

SRC_DIR = './tmTheme'
DIST_DIR = './themeCSS'
filenames = [filename for filename in listdir(SRC_DIR) if isfile(join(SRC_DIR, filename))]

In [6]:
def parse_array(array_node):
    result = []
    for child in array_node:
        if child.tag == 'dict':
            result.append(parse_dict(child))
    return result

def parse_dict(dict_node):
    result = {}
    key = None
    for child in dict_node:
        if child.tag == 'key':
            key = child.text
        elif child.tag == 'string':
            result[key] = child.text
        elif child.tag == 'dict':
            result[key] = parse_dict(child)
        elif child.tag == 'array':
            result[key] = parse_array(child)
    return result

In [7]:
for filename in filenames:
    filepath = join(SRC_DIR, filename)
    tree = ET.parse(filepath)
    root = tree.getroot() # plist

    assert(root.tag == 'plist')
    d = parse_dict(root[0])
    
    settings = d['settings']

    result = ''

    for value in settings:
        if 'name' not in value:
            result += 'body {\n'
            if 'foreground' in value['settings']:
                    result += '    color: ' + value['settings']['foreground'] + ';\n'
            if 'background' in value['settings']:
                result += '    background-color: ' + value['settings']['background'] + ';\n'
            result += '} \n'
            continue
        if 'scope' not in value:
            continue
        scope = value['scope'].replace('.', '-')
        ss = scope.split(',')
        for s in ss:
            s = s.strip()
            result += '.' + 'jc-' + s + ' {' + '\n'
            if 'settings' in value:
                if 'foreground' in value['settings'] and value['settings']['foreground']:
                    result += '    color: ' + value['settings']['foreground'] + ';\n'
                if 'background' in value['settings'] and value['settings']['background']:
                    result += '    background-color: ' + value['settings']['background'] + ';\n'
            result += '} \n'
    
    target_path = join(DIST_DIR, filename) + '.css'
    with open(target_path, 'w') as f:
        f.write(result)